In [669]:
# pdf_path = "test/pdf/dc/form/Ceridian-2-EC-SDS-v2.1.pdf"
# output_pdf_path = "test/target/dc/form/Ceridian-2-EC-SDS-v2.1.pdf"

pdf_path = "test/pdf/dc/form/acuron-label.pdf"
output_pdf_path = "test/target/dc/form/acuron-label.pdf"

# pdf_path = "test/pdf/dc/form/Blue_Book_2023_web.pdf"
# output_pdf_path = "test/target/dc/form/Blue_Book_2023_web.pdf"

In [670]:
import fitz
import json
import pandas as pd
import numpy as np
from pprint import pprint
from collections import defaultdict
from typing import Iterable


In [671]:
doc = fitz.open(pdf_path)

In [672]:
doc.page_cropbox(0)

Rect(0.0, 0.0, 612.0, 792.0)

In [673]:
# doc mid point
page_x_mid = doc.page_cropbox(0)[2] / 2
page_y_mid = doc.page_cropbox(0)[3] / 2
page_y_mid

396.0

In [674]:
left_edges = defaultdict(lambda: 0)

In [675]:

def get_edges(pages):
    left_edges = defaultdict(lambda: 0)
    right_edges = defaultdict(lambda: 0)
    top_edges = defaultdict(lambda: 0)
    bottom_edges = defaultdict(lambda: 0)

    if isinstance(pages, fitz.Page):
        pages = [pages]

    for page in pages:
        blocks = page.get_text("dict")["blocks"]

        for block in blocks:
            rect = fitz.Rect(block["bbox"])

            l = round(rect.x0)
            r = round(round(rect.x1) / 10) * 10
            t = round(rect.y0)
            b = round(round(rect.y1) / 10) * 10

            height = rect.height
            left_edges[l] += height if l < page_x_mid else 0
            right_edges[r] += height if r > page_x_mid else 0

            width = rect.width
            top_edges[t] += width if t < page_y_mid else 0
            bottom_edges[b] += width if b > page_y_mid else 0

    left_edge = max(left_edges, key=left_edges.get)
    right_edge = max(right_edges, key=right_edges.get)
    top_edge = max(top_edges, key=top_edges.get)
    bottom_edge = max(bottom_edges, key=bottom_edges.get)
    
    pprint(bottom_edges)

    return left_edge, right_edge, top_edge, bottom_edge

In [676]:
global_left_edge, global_right_edge, global_top_edge, global_bottom_edge = get_edges(doc)
global_left_edge
global_right_edge
global_top_edge
global_bottom_edge

defaultdict(<function get_edges.<locals>.<lambda> at 0x121698220>,
            {130: 0,
             150: 0,
             160: 0,
             170: 0,
             180: 0,
             190: 0,
             200: 0,
             210: 0,
             220: 0,
             230: 0,
             240: 0,
             250: 0,
             260: 0,
             270: 0,
             280: 0,
             290: 0,
             300: 0,
             310: 0,
             320: 0,
             330: 0,
             340: 0,
             350: 0,
             360: 0,
             370: 0,
             380: 0,
             390: 0,
             400: 2507.770347595215,
             410: 2503.578727722168,
             420: 1770.943016052246,
             430: 4151.957702636719,
             440: 2563.889488220215,
             450: 3958.190589904785,
             460: 3646.878288269043,
             470: 1688.6698379516602,
             480: 3834.7780380249023,
             490: 2371.4856491088867,
             5

430

In [677]:
for page in doc:
    blocks = page.get_text("dict")["blocks"]

    left_edge, right_edge, top_edge, bottom_edge = get_edges(page)
    left_edge = min(left_edge, global_left_edge)
    right_edge = max(right_edge, global_right_edge)
    top_edge = min(top_edge, global_top_edge)
    bottom_edge = max(bottom_edge, global_bottom_edge)

    rect = fitz.Rect(left_edge, 0, left_edge + 1, 100000)
    page.draw_rect(rect, color=[0.4,0.8,0.6], width=1.2, overlay=True, stroke_opacity=0.7)

    rect = fitz.Rect(right_edge, 0, right_edge + 1, 100000)
    page.draw_rect(rect, color=[0.4,0.8,0.6], width=1.2, overlay=True, stroke_opacity=0.7)

    rect = fitz.Rect(0, top_edge, 100000, top_edge + 1)
    page.draw_rect(rect, color=[0.4,0.8,0.6], width=1.2, overlay=True, stroke_opacity=0.7)

    rect = fitz.Rect(0, bottom_edge, 100000, bottom_edge + 1)
    page.draw_rect(rect, color=[0.4,0.8,0.6], width=1.2, overlay=True, stroke_opacity=0.7)

    for block in blocks:
        rect = fitz.Rect(block["bbox"])

        page.draw_rect(
            rect,
            color=[0,0,1],
            fill=None,
            width=1.2,
            overlay=True,
        )
        

defaultdict(<function get_edges.<locals>.<lambda> at 0x1082163e0>,
            {130: 0,
             220: 0,
             240: 0,
             280: 0,
             290: 0,
             440: 330.737060546875,
             500: 356.96678924560547,
             510: 356.4998779296875,
             520: 356.22296142578125,
             540: 351.8885192871094,
             600: 248.17919921875,
             610: 258.2440185546875,
             660: 416.3531494140625})
defaultdict(<function get_edges.<locals>.<lambda> at 0x1082177e0>,
            {160: 0,
             200: 0,
             230: 0,
             260: 0,
             310: 0,
             320: 0,
             370: 0,
             400: 138.5601806640625,
             430: 232.87126922607422,
             450: 431.42488861083984,
             470: 140.65152740478516,
             480: 203.29463958740234,
             550: 424.96053314208984,
             560: 181.57415771484375,
             590: 431.4151306152344,
             610

In [678]:
doc.save(output_pdf_path)